In [1]:
import tensorflow as tf
from tensorboard.plugins import projector

import os
import json

In [2]:
%load_ext tensorboard

In [3]:
input_vocab_path = '../models/checkpoints/seq2seq-best-july-8/input_vocab_index.json'
with open(input_vocab_path) as f:
    input_vocab_index = json.load(f)

In [4]:
log_dir = '../reports/embeddings/attempt-3/'
os.makedirs(log_dir, exist_ok=True)

In [5]:
max_vocab_size = 7000

In [6]:
with open(os.path.join(log_dir, 'metadata.tsv'), 'w') as f:
    word_index = input_vocab_index.keys()
    for word in list(word_index)[:max_vocab_size]:
        f.write(f'{word}\n')

    # Fill in the rest of the labels with "unknown"
    for unknown in range(1, len(word_index) - max_vocab_size):
        f.write(f'unknown #{unknown}\n')

In [7]:
!tail ../reports/embeddings/attempt-3/metadata.tsv

unknown #17256
unknown #17257
unknown #17258
unknown #17259
unknown #17260
unknown #17261
unknown #17262
unknown #17263
unknown #17264
unknown #17265


In [8]:
!wc ../reports/embeddings/attempt-3/metadata.tsv

 24265  41530 298959 ../reports/embeddings/attempt-3/metadata.tsv


In [9]:
checkpoint = tf.train.Checkpoint()

In [10]:
checkpoint.restore(tf.train.latest_checkpoint(log_dir))

In [11]:
tf.train.list_variables(log_dir)

[('_CHECKPOINTABLE_OBJECT_GRAPH', []),
 ('decoder/attention/V/bias/.ATTRIBUTES/VARIABLE_VALUE', [1]),
 ('decoder/attention/V/bias/.OPTIMIZER_SLOT/optimizer/m/.ATTRIBUTES/VARIABLE_VALUE',
  [1]),
 ('decoder/attention/V/bias/.OPTIMIZER_SLOT/optimizer/v/.ATTRIBUTES/VARIABLE_VALUE',
  [1]),
 ('decoder/attention/V/kernel/.ATTRIBUTES/VARIABLE_VALUE', [768, 1]),
 ('decoder/attention/V/kernel/.OPTIMIZER_SLOT/optimizer/m/.ATTRIBUTES/VARIABLE_VALUE',
  [768, 1]),
 ('decoder/attention/V/kernel/.OPTIMIZER_SLOT/optimizer/v/.ATTRIBUTES/VARIABLE_VALUE',
  [768, 1]),
 ('decoder/attention/W1/bias/.ATTRIBUTES/VARIABLE_VALUE', [768]),
 ('decoder/attention/W1/bias/.OPTIMIZER_SLOT/optimizer/m/.ATTRIBUTES/VARIABLE_VALUE',
  [768]),
 ('decoder/attention/W1/bias/.OPTIMIZER_SLOT/optimizer/v/.ATTRIBUTES/VARIABLE_VALUE',
  [768]),
 ('decoder/attention/W1/kernel/.ATTRIBUTES/VARIABLE_VALUE', [768, 768]),
 ('decoder/attention/W1/kernel/.OPTIMIZER_SLOT/optimizer/m/.ATTRIBUTES/VARIABLE_VALUE',
  [768, 768]),
 ('decod

In [12]:
tf.train.load_variable(log_dir, 'encoder/embedding/embeddings/.ATTRIBUTES/VARIABLE_VALUE').shape

(7000, 128)

In [13]:
# Set up config
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
# The name of the tensor will be suffixed by `/.ATTRIBUTES/VARIABLE_VALUE`
embedding.tensor_name = 'encoder/embedding/embeddings/.ATTRIBUTES/VARIABLE_VALUE'
embedding.metadata_path = 'metadata.tsv'
projector.visualize_embeddings(log_dir, config)

In [14]:
%tensorboard --logdir ../reports/embeddings/attempt-3/